# Progress Report Summary

## Outline of work done

* ### Data preparation 
* ### Metrics we care about
* ### Initial runs using logistic regression
* ### Look at sampling techniques
* ### Naive validation using test_train_split
* ### Custom cross validation
* ### Running on all classifiers, for all sampling techniques
* ### Baseline models evaluation
* ### CNN work - data and model prep
* ### CNN model 1 running and cross-val
* ### CNN model 1 evaluation

## Data Preparation

I started baseline work by looking at the data and preparing it. Firstly I explored the structure of the data and viewed the imbalance of the two classes. Then, I prepared data by normalising the 'Amount' column of the data to range between -1 and 1 like the rest of the data and then I dropped the 'Time' column completely. I did this in order to not work with time series data and to focus solely on classification using the other 29 features. 

#### Normalise the Amount column and drop the Time column

 Here we normalise the Amount column to align better with the other features in the data. This is to eliminate the influence of one feature over another and also to aid gradient descent learning. This is done by adding a new column that is the normalised version of the current Amount column and then dropping the old one. 

 We also drop the Time column completely. The reason for this is that the time data is simply a timestamp and for the purposes of obtaining baseline classification results, we can omit this and work solely on classification from the 29 other features, instead of introducing time series, temporal classification. Also, as there will be no feature engineering done here, timestamps over 2 days will not be useful.

---

#### Feature Engineering

In a typical data science machine learning workflow, one would usually explore the data and engineer extra features based on domain knowledge of the data and insights into what else might be useful. In this case however as the data is post-PCA and anonymised, it makes no sense to engineer further features. There could be some potential ideas such as engineering features based on transactions in a small space of time or large transaction amounts etc but since this is for baseline work only and that the data is only for a 2 day period, we wont engineer any features  or include the time column in our training.


## Metrics we care about

#### Problem with Accuracy as a metric
In a lot of machine learning projects, Accuracy is a common metric to use to evaluate the perform of classification problems. However in our case this will not work. 

##### Even if we completely classified everything we see as non fraudluent, the model would still be deemed to be 99.83% accurate, which is incredibly high. 

#### F1-score:
F1-score is the harmonic average of precision and recall. We can define precision as intuitively the ability of the classifier not to label as positive, a sample that is negative. Similarly we define recall as intuitively the ability of the classifier to find all the positive samples.

#### So why do we care about F1 score? ...

##### Recall - Catching fraudulent transactions
As described above, our model can be 99.83% accurate whilst not catching ANY fraud. Recall determines the amount of the positive class we correctly classify. So we are interested in this as the 'primary' function of the work we are doing: to classify fraud.

##### Precision - Not missclassifying non-fraudulent transactions
Of course, a model that just classifies everything as fraud is not usable. We want this to be useful and hence we dont want this to happen. So precision is also important.

#### Context of the bank
In the context of banks and how these metrics add or lose value to them, we indeed care about F1 score. As you can see from the results table for SMOTE, LinearSVC has the highest Recall score, which means it is great at finding true fraudulent cases. However, it's precision is very low at 0.06. This essentially means that the classifier performs badly when it comes to predicting some non-fraudulent data and falsly labelling them as fraud. Why is this bad? This is bad because the loses the bank money and gives customers a bad experience. If we have bad precision then we falsly classify as fraud a lot and we freeze customer cards and accounts and send them a text to say we believe there is fraud etc. Only to ultimately verify that everything is benign and reverse the situation. This is very bad and gives a bad impression for the customer, who may indeed change bank or lose faith in the bank's intelligence systems.

#### Hence, we care about F1 score, which is the balance between these two metrics. Recall: being able to catch true frauds and Precision: being able to correctly classify and reduce the number of false positives. Even when we have balanced the dataset through resampling, these metrics are still the quantities we care about.

## Custom Cross Validation

When cross validating using resampling techniques, you have to be careful you are not introducing any bias to the model along the way. These are some of the points addressed in the way cross validation is performed in this project:

### - In the case of oversampling and Smote, preserving a test set and resampling only training data

It is important we preserve a 'pure' real-life sample of the data for testing, that is not manipulated by resampling. This is important to test the generalisability of the model and ensures that we don't introduce any bias by using resampled data in our testing set. The whole point of validating the model is to test it's performance on unseen data, after all.

### - Oversampling inside the cross validation loop

A typical approach would perhaps be to simply oversample the data and then pass this onto cross validation. This will introduce bias by allowing oversampled minority class data points to 'leak' into the validation test set of the CV loop. This will therefore give a higher result as the model has already seen these data points. In the case of Oversampling the minority class datapoints are simply duplicated so it is especially important here. This is not the case for SMOTE, which creates synthetic data points but as described before, it is still important to preserve a 'pure' test set.

To visualise this, see the following:

![caption](cv-oversample-1.jpg)

![caption](cv-oversample-2.jpg)

Diagrams from https://www.marcoaltini.com/blog/dealing-with-imbalanced-data-undersampling-oversampling-and-proper-cross-validation.

### - Test_Train_Split vs Custom cross_val_score using KFold

This concerns the underlying approach for the cross validation loop. We have have already seen we can use sklearn's test_train_split function to split the data into train and test portions. We can then oversample our training data, fit the model and make predictions using the test set. 

#### Approach 1 would be to simply do this multiple times and average the results. 

This approach, however, will give better results than expected (which will be shown in a later section, I go on to implement this version for comparison). The reason for this is due to the nature of test_train_split and it's randomness. 

#### Problems with this method:
Test_train_split allows you to randomly split your data, by giving a parameter that specifies the ratio. However as the split is random, it is likely that there will be overlap in the CV iterations as to which data points are put in the test set. In other words, values selected during one iteration, could be selected again during another iteration.

The consequences of this means that the model may not be exposed to particular portions of the data whereby it does not generalise well and we are not capturing that in our results. Also, It is not making maximal use of the data we have.

#### Approach 2 - KFOLD

Kfold is a well known cross validation technique whereby instead of random splitting of the data, the data is split equally into N folds. Each iteration then uses N-1 folds for training and the Nth fold for validation. This means that the classifier gets tested on all parts of our data and will capture all variations on performance. It also means we make maximal use of our data. 

#### Therefore the approach that is used is:
* Original dataset -> Cross-Val loop
* Cross-Val Loop: N interations of KFOLD technique
* Each KFOLD iteration: Oversample the training folds, every time and validate on the preserved test fold

___

### Baseline Models Evaluation
---

### Original vs Under-Sampled

The results certainly show that undersampling performs extremely well. However when we intuitively think about why this is so, it is perhaps not the wise approach. When under-sampling we have reduced the amount of information we have from over 200,000 real life examples (albeit benign transactions) and brought this down to merely <500. 
By doing this, as mentioned, we lose a *lot* of information that the classifier could learn from and hence become more generalisable. It is likely the case that we just overfit to the small dataset.

To this end, it would appear that perhaps no sampling is better here. The original data performed fairly well across the board of classifiers, with precision being stronger as the majority. 

### Oversample vs SMOTE

As seen, the results for Oversampling are marginally higher than that of SMOTE but due to the understanding of how the algorithms work and how we resample inside the cross validation loop, it is easy to understand why these results are likely to be biased.

Oversampling simply duplicates datapoints randomly, so there is a lot of redundant data floating around. This means it is likely that during the CV process, the testing fold will likely contain duplicate data as in some of the training folds and therefore we have a 'leakage' of test data. This would explain why Oversampling appears to achieve better results.

For this reason, taking the results of SMOTE is preferred, as it is more 'true'. This is because SMOTE uses K Nearest Neigbours to pick a nearby datapoint at random and then it randomly extrapolates this point in either the positive of negative direction by an amount in the range [0,1]. Effectively, this creates new data points which is a lot better to train on than duplicate information.

### Original vs SMOTE

The question then begs, is SMOTE worth it at all? Given that the original dataset achieves quite well in comparison. Well, it depends what we care about most. If we care about Recall and hence catching fraudluent transactions, then SMOTE has an advantage here. Taking the best achieving classifier, that is RandomForest, we can see that by using SMOTE we have a considerable margin on Recall, at the expense of some precision, but maintaining a marginally hgiher F1 score overall. So, by considering the classifier that appears to be suited for the problem at hand, SMOTE allows us to achieve higher than the original dataset.

### Test_Train_Split vs Custom cross_val_score using KFold
#### How a difference in splitting can influence results

To represent the importance of ensuring all of the data is used to validate the model (using KFold), we look at the results of using test_train_split to split the data and then resampling the training data only (to preserve test data) and then averaging this n times. This variation in results show that we have to be careful in cross validating correctly in order to achieve the most true results.

### Best classifier

An overall majority classifier that appears to be most suited to this problem is the Random Forest Classifier. This classifier can maintain precision the best when resampling the data, to leave a respectable F1-score. Taking this classifier as the most suited and fine tuning it to try and squeeze the best possible result gave an F1 score of 0.827025, using SMOTE. This can be used as an estimate of the best value we can achieve from baseline work, for use in comparisons with later models. 




## CNN model 1 

### The approach:

#### Without time feature
#### Pass 1x29 vectors into a convolutional layer, with kernel size 29, with some D number of filters
#### Add extra conv and dense layer to the model to see the effect

I created a function for returning the CNN model, which we can use as a functional parameter to the custom cross validation we perform. 

The first of these is with a single convolutional layer and a dense layer after flattening. In addition there is a 2 node categorical output layer. 
The second is similar but with an extra Conv layer and an extra dense layer. 

The data prepared for this is smote data, as before in the baseline models and the method of custom cross validation is the same, just slightly adapted to handle our create_model functions.

### Results and Evaluation

see appendix for results.

We see that in this simple CNN approach, we do not meet the best we have seen amongst baseline models. However, looking a bit more specifically, we see that Recall scores are comparable, but without the drastic low in precision scores, like we had in some baseline models. 

Comparing specifically with the (tuned) Random Forest classifier:
We see that the Recall is approximately the same, it is the precision that is lower.

Comparing with the Smote results across all baseline models we can see that our CNN models are higher in F1-score except the RF classifier. In particular the precision scores are comparable to the second best performing classifier the MLP. 

In going from CNN1 to CNN1.2, we see that the major performance increase comes from raising the precision score, resulting in a higher F1 overall. This means that our approach of adding an extra conv layer and dense layer helped to improve precision of the model. The training time however, obviously took longer, almost double the time.

## Ongoing and next steps

### CNN model 2 set up done
### Currently preparing data:
#### Have appropriately split train/test to preserve time 
#### Need to pad/trim appropriately so there are even batches for the model
### Need to encorporate into custom cross val function and train

# Results Appendix

In [ ]:
 ''' Accounting properly for sampling in CV
 
 ORIGINAL
 ==============================
Cross validation training results: 
                        F1 Score  Precision    Recall   Training Time
Classifier                                                           
KNeighborsClassifier    0.773953   0.834210  0.733740 00:00:00.672486
LinearSVC               0.702664   0.911093  0.609756 00:00:44.615623
DecisionTreeClassifier  0.648748   0.584061  0.747967 00:00:12.705187
RandomForestClassifier  0.789572   0.867006  0.737805 00:00:11.946860
MLPClassifier           0.740725   0.789986  0.701220 00:00:10.611695
GaussianNB              0.114077   0.061255  0.833333 00:00:00.110761
==============================

UNDER
==============================
Cross validation training results: 
                        F1 Score  Precision    Recall   Training Time
Classifier                                                           
KNeighborsClassifier    0.929551   0.978015  0.886179 00:00:00.000986
LinearSVC               0.908181   0.955016  0.865854 00:00:00.016134
DecisionTreeClassifier  0.900284   0.910581  0.890244 00:00:00.009997
RandomForestClassifier  0.915486   0.952169  0.882114 00:00:00.040110
MLPClassifier           0.916529   0.956296  0.880081 00:00:00.545562
GaussianNB              0.900877   0.965324  0.845528 00:00:00.001272
==============================

OVER
==============================
Cross validation training results: 
                        F1 Score  Precision    Recall   Training Time
Classifier                                                           
KNeighborsClassifier    0.634392   0.563856  0.798780 00:00:01.675460
LinearSVC               0.117779   0.063179  0.871951 00:01:19.514793
DecisionTreeClassifier  0.649036   0.661330  0.664634 00:00:07.959666
RandomForestClassifier  0.801992   0.884864  0.745935 00:00:10.799792
MLPClassifier           0.658083   0.595006  0.774390 00:00:24.078549
GaussianNB              0.100591   0.053461  0.855691 00:00:00.198887
==============================

SMOTE
==============================
Cross validation training results: 
                        F1 Score  Precision    Recall   Training Time
Classifier                                                           
KNeighborsClassifier    0.500628   0.375985  0.831301 00:00:01.722399
LinearSVC               0.116943   0.062724  0.867886 00:01:21.129306
DecisionTreeClassifier  0.441011   0.325126  0.711382 00:00:23.395320
RandomForestClassifier  0.806464   0.833590  0.794715 00:00:21.880070
MLPClassifier           0.709039   0.708030  0.725610 00:00:24.371739
GaussianNB              0.107311   0.057277  0.855691 00:00:00.193990
==============================
 
 '''

In [ ]:
''' Naive averaging, using test_train_split only. 
    Oversampling training split and preserving test set, averaging over 3 runs
    
SUMMARY OF RESULTS (AVG over three random iterations)
=====================================================================
                        F1 Score  Precision    Recall   Training Time
Classifier                                                           
KNeighborsClassifier    0.611444   0.476876  0.853809 00:00:01.758558
LinearSVC               0.119385   0.063951  0.896587 00:01:24.125692
DecisionTreeClassifier  0.536858   0.411079  0.774736 00:00:23.579666
RandomForestClassifier  0.846437   0.879453  0.816384 00:00:23.179558
MLPClassifier           0.750672   0.704481  0.807383 00:00:27.263501
GaussianNB              0.109543   0.058463  0.870450 00:00:00.213850
=====================================================================
'''

In [ ]:
''' Tuned Random Forest Classifier, the best performer:

['RandomForestClassifier', 0.827025, 0.853782, 0.813008, datetime.timedelta(0, 419, 267422)] 

F1 SCORE = 0.827025

'''

In [ ]:
''' CNN APPROACH 1 RESULTS
               F1 Score  Precision    Recall   Training Time
Name                                                        
CNN Model 1    0.696680   0.628969  0.821138 00:01:42.882115
CNN Model 1.2  0.745591   0.698229  0.817073 00:03:42.131883

'''